Импортируем необходимые библиотеки

In [2]:
!pip install transformers

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

tqdm.pandas()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.6 MB/s eta 0:00:00


Монтируем Google Drive для загрузки датасета

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Задания/Assignment02'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Задания/Assignment02


Загружаем датасет

In [4]:
df = pd.read_csv("rusentitweet_full.csv")

Функция для изменения меток классов на целые числа

In [5]:
def change_label(label):
    if label == 'skip':
        return 0
    elif label == 'positive':
        return 1
    if label == 'neutral':
        return 2
    if label == 'speech':
        return 3
    if label == 'negative':
        return 4

df['label'] = df['label'].apply(change_label)

Функция, удаляющая из текста имена пользователей, HashTag'и, ссыллки и эмодзи, а также приводящая текст к нижнему регистру

In [6]:
import regex as re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\\x\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

Подготавливаем датасет

In [7]:
df['text'] = df['text'].apply(clean_text)

Токенизируем текст

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

def tokenize(text):
    res = tokenizer.encode_plus(
        text,
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors='pt'
    )
    return pd.Series([res['input_ids'], res['attention_mask']])

df[['input_ids', 'attention_mask']] = df['text'].progress_apply(tokenize)

  0%|          | 0/13392 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Разбиваем датасет на тренировочную и тестовую выборки

In [9]:
test_size = 0.2
batch_size = 32
train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    # shuffle=True,
    random_state=42,
    stratify=df['label'].values
)
train_set = TensorDataset(
    torch.cat(list(train_df["input_ids"].values), dim = 0),
    torch.cat(list(train_df["attention_mask"].values), dim = 0),
    torch.tensor(train_df["label"].values)
)
test_set = TensorDataset(
    torch.cat(list(test_df["input_ids"].values), dim = 0),
    torch.cat(list(test_df["attention_mask"].values), dim = 0),
    torch.tensor(test_df["label"].values)
)
train_dataloader = DataLoader(
    train_set, 
    batch_size=batch_size, 
    shuffle=True
)
test_dataloader = DataLoader(
    test_set, 
    batch_size=batch_size, 
    shuffle=True
)

Выбираем устройство в зависимости от текущей конфигурации

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Модель

In [11]:
bert_model = BertForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased-sentence', 
    num_labels=5,
)
bert_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

Функция для тренировки и тестирования модели

In [12]:
epochs = 2
optimizer = AdamW(bert_model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

def train(model, train_dataloader, test_dataloader, epochs, optimizer, scheduler):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_f1 = 0
        for batch in tqdm(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs[0]
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        train_loss /= len(train_dataloader)
        train_f1 /= len(train_dataloader)
        print(f"Epoch: {epoch+1}")
        print(f"Train Loss: {train_loss}")
        print(f"Train F1: {train_f1}")
        print("Evaluating...")
        model.eval()
        test_loss = 0
        test_f1 = 0
        for batch in tqdm(test_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
            loss = outputs[0]
            logits = outputs[1]
            test_loss += loss.item()
            preds = torch.argmax(logits, dim=1).flatten()
            test_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
        test_loss /= len(test_dataloader)
        test_f1 /= len(test_dataloader)
        print(f"Test Loss: {test_loss}")
        print(f"Test F1: {test_f1}")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Обучаем модель и экспортируем ее в файл

In [13]:
train(bert_model, train_dataloader, test_dataloader, epochs, optimizer, scheduler)

torch.save(bert_model.state_dict(), 'bert_model.pt')

  0%|          | 0/335 [00:00<?, ?it/s]

Epoch: 1
Train Loss: 1.0204902985202733
Train F1: 0.0
Evaluating...


  0%|          | 0/84 [00:00<?, ?it/s]

Test Loss: 0.9177265635558537
Test F1: 0.610729417869304


  0%|          | 0/335 [00:00<?, ?it/s]

Epoch: 2
Train Loss: 0.7601118819927102
Train F1: 0.0
Evaluating...


  0%|          | 0/84 [00:00<?, ?it/s]

Test Loss: 0.9073539021469298
Test F1: 0.6182437810688961


Wrapper для использования модели

In [14]:
import os

def predict(text) -> str:
    if clean_text is None:
        raise Exception('clean_text function is not defined')
    if tokenize is None:
        raise Exception('tokenize function is not defined')
    text = clean_text(text)
    input_ids, attention_mask = tokenize(text)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    if not os.path.exists('models/bert_model.pt'):
        raise Exception('the model is not trained yet')
    bert_model.load_state_dict(torch.load('models/bert_model.pt'))
    bert_model.to(device)
    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
    logits = outputs[0]
    preds = torch.argmax(logits, dim=1).flatten()
    if preds.item() == 0:
        return 'skipped'
    elif preds.item() == 1:
        return 'positive'
    elif preds.item() == 2:
        return 'neutral'
    elif preds.item() == 3:
        return 'speech'
    elif preds.item() == 4:
        return 'negative'
    else:
        return 'unknown'

def predict_with_input(text) -> (str, str):
    res = predict(text)
    return f'{text} -> {res}\n'